In [ ]:
# Configuration flags
#SAMPLE_ENABLE = False
#SAMPLE_SIZE = 20000
#MIMIC_PATH = "mimic-db-short"
#
#
#
#print("🏥 Loading MIMIC-III data...")
#
## Step 1: First, find ICUSTAY_IDs that have ALL required vital signs
#print("📂 Loading CHARTEVENTS...")
#
#
#chartevents_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/CHARTEVENTS.csv")
#print("✅ Loaded CHARTEVENTS from parquet")
#
#
#
#
## Step 2: Load ICUSTAYS 
#print("\n📂 Loading and filtering ICUSTAYS...")
#icustays_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ICUSTAYS.csv")
#
#
#
## Step 3: Apply sampling if enabled
#if SAMPLE_ENABLE:
#    print(f"🎯 Sampling {SAMPLE_SIZE} ICU stays...")
#    icustays_df = icustays_df.limit(SAMPLE_SIZE)
#    icustays_df.cache()
#    actual_sample_size = icustays_df.count()
#    print(f"✅ Final sample: {actual_sample_size} ICU stays")
#else:
#    icustays_df.cache()
#    actual_sample_size = icustays_df.count()
#
#    
#    
## Step 4: Create efficient lookup tables
#print("📋 Creating ID lookup tables...")
#icu_lookup = icustays_df.select("ICUSTAY_ID").distinct().cache()
#hadm_lookup = icustays_df.select("HADM_ID").distinct().cache()
#subject_lookup = icustays_df.select("SUBJECT_ID").distinct().cache()
#
#icu_lookup.count()  # Trigger caching
#hadm_lookup.count()
#subject_lookup.count()
#
## Step 5: Load other tables with optimized joins
#print("📂 Loading PATIENTS table...")
#patients_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/PATIENTS.csv")
#patients_df = patients_df.join(broadcast(subject_lookup), "SUBJECT_ID", "inner")
#
#print("📂 Loading ADMISSIONS table...")
#admissions_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ADMISSIONS.csv")
#admissions_df = admissions_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")
#
#print("📂 Loading DIAGNOSES_ICD table...")
#diagnoses_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/DIAGNOSES_ICD.csv")
#diagnoses_df = diagnoses_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")
#
## Step 6: Load and filter CHARTEVENTS efficiently
#print("📂 Loading CHARTEVENTS table... [FILTERING BY ICUSTAY_ID]")
#chartevents_df = chartevents_df \
#    .select("ICUSTAY_ID", "CHARTTIME", "ITEMID", "VALUE", "VALUEUOM", "VALUENUM") \
#    .join(broadcast(icu_lookup), "ICUSTAY_ID", "inner")
#
## Step 7: Load LABEVENTS
#print("📂 Loading LABEVENTS table... [FILTERING BY HADM_ID]")
#
#labevents_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/LABEVENTS.csv")
#
#
#labevents_df = labevents_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")
#
## Final summary
#print("\n✅ Data loading complete!")
#print(f"📊 ICUSTAYS: {icustays_df.count():,} rows")
#print(f"📊 PATIENTS: {patients_df.count():,} rows") 
#print(f"📊 ADMISSIONS: {admissions_df.count():,} rows")
#print(f"📊 DIAGNOSES_ICD: {diagnoses_df.count():,} rows")
#print(f"📊 CHARTEVENTS (filtered): {chartevents_df.count():,} rows")
#print(f"📊 LABEVENTS (filtered): {labevents_df.count():,} rows")
#print(f"\n⏰ Data loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Tentativa #1

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier  # or any other algorithm
from pyspark.ml.evaluation import BinaryClassificationEvaluator  # or MulticlassClassificationEvaluator
from pyspark.sql.functions import coalesce, lit

## 0. Initialize Spark Session

In [2]:
spark = SparkSession.builder \
    .appName("BigDataMLProject") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 10:32:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Load Data locally

In [3]:
DATA_PATH = "./mimic-db-short"

items = spark.read.csv(f"{DATA_PATH}/D_ITEMS.csv", header=True, inferSchema=True)
procedures = spark.read.csv(f"{DATA_PATH}/PROCEDURES_ICD.csv", header=True, inferSchema=True)

chartevents = spark.read.csv(f"{DATA_PATH}/CHARTEVENTS.csv", header=True, inferSchema=True)
patients = spark.read.csv(f"{DATA_PATH}/PATIENTS.csv", header=True, inferSchema=True)
admissions = spark.read.csv(f"{DATA_PATH}/ADMISSIONS.csv", header=True, inferSchema=True)
diagnoses = spark.read.csv(f"{DATA_PATH}/DIAGNOSES_ICD.csv", header=True, inferSchema=True)
icustays = spark.read.csv(f"{DATA_PATH}/ICUSTAYS.csv", header=True, inferSchema=True)
inputevents = spark.read.csv(f"{DATA_PATH}/INPUTEVENTS_MV.csv", header=True, inferSchema=True)
labevents = spark.read.csv(f"{DATA_PATH}/LABEVENTS.csv", header=True, inferSchema=True)

print("Data Loaded!")


Data Loaded!


## 2. Data Preprocessing

The tables were chose based on its properties regarding a persons' illness, bodily atributes or gravity of the situation. The tables were:
* ChartEVents
* Admissions
* Patients
* Diagnoses
* ICUStays
* InputEvents_MV
* Procedures

### A. Feature Engineering

The majoriry of the atributes are not useful to predict the duration of a ICU stay, therefore, for a initial analysis, we will choose the ones we consider relevant.

First of all, we create temporary views of all tables into a SQL-like table.

In [4]:
chartevents.createOrReplaceTempView("chartevents")
patients.createOrReplaceTempView("patients")
admissions.createOrReplaceTempView("admissions")
diagnoses.createOrReplaceTempView("diagnoses")
icustays.createOrReplaceTempView("icustays")
inputevents.createOrReplaceTempView("inputevents")
labevents.createOrReplaceTempView("labevents")
procedures.createOrReplaceTempView("procedures")

25/06/02 10:33:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SO QUEREMOS PESSOAS QUE TENHAM ICUSTAY_ID PQ: given a person admitted int he icu i have to predict for how long they are gonna be there, therefore in pre processing i will ignore the data from people that were never in the icu. The most efficient approach is to create temporary views that pre-filter the data before joining tables.

In [5]:
spark.sql("CREATE OR REPLACE TEMPORARY VIEW icu_filtered AS SELECT * FROM icustays WHERE icustay_id IS NOT NULL")
spark.sql("CREATE OR REPLACE TEMPORARY VIEW chart_filtered AS SELECT * FROM chartevents WHERE icustay_id IS NOT NULL")
spark.sql("CREATE OR REPLACE TEMPORARY VIEW input_filtered AS SELECT * FROM inputevents WHERE icustay_id IS NOT NULL")

DataFrame[]

To get procedures done to icu patients

In [6]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW proc_filtered AS
SELECT 
    p.icd9_code,
    icu.icustay_id
FROM procedures p
JOIN icustays icu ON p.hadm_id = icu.hadm_id
""")

DataFrame[]

To handle labevents time to be only during the icustay: get only one row by labevent, the first valid (not null result) the person did when enterying the icu. To predcit the legth someone will stay, when they first enter, we will only have the first test, so we will train the machine learning using this first value.

In [8]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW lab_filtered AS
SELECT 
    lab.subject_id,
    lab.hadm_id,
    lab.itemid,
    lab.valuenum,
    lab.valueuom,
    icu.icustay_id,
    lab.charttime 
FROM labevents lab
JOIN admissions adm ON lab.hadm_id = adm.hadm_id
JOIN icustays icu ON adm.hadm_id = icu.hadm_id
WHERE 
    lab.charttime BETWEEN icu.intime AND icu.outtime
    AND lab.hadm_id IS NOT NULL
""")


spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW lab_filtered2 AS
SELECT 
    subject_id,
    hadm_id,
    itemid,
    FIRST_VALUE(valuenum) IGNORE NULLS OVER (
        PARTITION BY icustay_id, itemid 
        ORDER BY charttime
    ) AS valuenum,
    icustay_id
FROM lab_filtered
""")


DataFrame[]

Then, we create a table, using a SQL query, with the relevant features for predicting the icu stay duration, and its own column (LOS from ICUStays).

In [17]:
df = spark.sql("""
    SELECT 
        icu.icustay_id,
        adm.admission_type,
        adm.admission_location,
        chart.itemid AS chart_itemid,
        chart.valuenum AS chart_value,
        chart.error,
        lab.itemid AS lab_itemid,
        lab.valuenum AS lab_value,
        diag.seq_num,
        diag.icd9_code as diagnose_code,
        input.itemid AS input_itemid,
        input.amount,
        input.rate,
        input.patientweight,
        proc.icd9_code as procedure_code,
        icu.first_careunit,
        icu.LOS
    FROM icu_filtered icu
    JOIN admissions adm ON icu.hadm_id = adm.hadm_id
    LEFT JOIN chart_filtered chart ON icu.icustay_id = chart.icustay_id
    LEFT JOIN lab_filtered2 lab ON icu.icustay_id = lab.icustay_id
    LEFT JOIN diagnoses diag ON icu.hadm_id = diag.hadm_id
    LEFT JOIN input_filtered input ON icu.icustay_id = input.icustay_id
    LEFT JOIN proc_filtered proc ON proc.icustay_id=icu.icustay_id
""")
df.show()

+----------+--------------+--------------------+------------+-----------+-----+----------+---------+-------+-------------+------------+-----------+-----------+-------------+--------------+--------------+------+
|icustay_id|admission_type|  admission_location|chart_itemid|chart_value|error|lab_itemid|lab_value|seq_num|diagnose_code|input_itemid|     amount|       rate|patientweight|procedure_code|first_careunit|   LOS|
+----------+--------------+--------------------+------------+-----------+-----+----------+---------+-------+-------------+------------+-----------+-----------+-------------+--------------+--------------+------+
|    207525|      ELECTIVE|PHYS REFERRAL/NOR...|      224001|       NULL|    0|     51301|     14.6|     12|        45829|      225158|0.033333335|  2.0000001|         99.2|          3961|          CSRU|1.2488|
|    207525|      ELECTIVE|PHYS REFERRAL/NOR...|      224001|       NULL|    0|     51301|     14.6|     12|        45829|      225158|0.033333335|  2.00000

merdas para justificar no relatorio:

Juntei as tabelas icu e admissions por admissao ao hospital e nao por pessoa, pq pode haver mais doq uma admissao por pessoa. Fiz outter join pelo msm motivo, mas dps tenho de testar isso com os dados todos e nao so com as tabelas parciais

na chartevents, juntei por icu stay, Each row associated with one ITEMID (e.g. 212) corresponds to an instantiation of the same measurement (e.g. heart rate) entao nao vale a pena por a coluna VALUEUOM is the unit of measurement, pq cada teste ja e feito numa unidade de medida, n vale a pena tar a dar mais do msm a ml. Vou por o erro pq se houve erro ent conta menos para a ml mas ig que e melhor q nada??? problema para a ml e nao para nos.

aqui cada chartevent, ou seja, analie/teste/raiox etc corresponde as uma linha, mais tarde temos de resolver este problema pq o objetivo e prever o tempo de internacao POR ICUSTAY_ID, ent deviamos ter apenas uma linha por admissao//icustay

na tabela dos diagnosticos SEQ_NUM provides the order in which the ICD diagnoses relate to the patient, o quao importante a doenca e no caso da pessoa, e ICD9_CODE contains the actual code corresponding to the diagnosis, ou seja a doenca id, por isso para dar join dou por paciente ou por admissao? aceito opinioes, por agr pus admissoes pq e oq pus no resto

input events dei join pela icustay ja que so queremos analisar pacientes que ja tenham estado. Each row associated with one ITEMID which corresponds to an instantiation of the same measurement (e.g. norepinephrine) AMOUNT - the amount of a drug or substance administered to the patient (ignorando se esta em ml dl ou l pelo motivo referido acima)


When predicting ICU length of stay (LOS), incorporating lab results can significantly improve your model's performance, but you're right that joining requires careful handling since LABEVENTS doesn't directly contain ICU stay IDs. juntar pelo ham id ou subject e dps escolher com base no tempo


the id is the icustays

justificar o left join:
For your use case, keep the LEFT JOINs but understand why:

    LEFT JOIN (icu → others) is correct because:

        You want all ICU stays (base table)

        You want to keep ICU stays even if they're missing some diagnoses/procedures

    Don't use FULL OUTER JOIN because:

        It would include diagnoses/procedures for non-ICU patients (if any exist)

        Could create NULL ICU stay records which you don't want

    Your current approach is good for:

        One row per combination of ICU stay + diagnosis + procedure

        Preserving all relationships

Example of What You'll Get

For a patient with:

    1 ICU stay

    4 diagnoses

    5 procedures

    10 lab results

Your query will produce 4 × 5 × 10 = 200 rows for this patient (all combinations).


FALTA:
altura / bmi do paciente
genero do paciente

Attribute/column documentation for the table created:

 1. Patient Identification
- **icustay_id**: Unique ICU stay identifier (primary key for ICU stays)

 2. Admission Information
- **admission_type**: Type of admission (ELECTIVE, EMERGENCY, URGENT, etc.)
- **admission_location**: Source of admission (TRANSFER FROM HOSPITAL, CLINIC REFERRAL, etc.)
- **first_careunit**: Initial ICU care unit (MICU, SICU, CSRU, etc.)
- **LOS**: Length of stay in ICU (in hours or days)

 3. Clinical Measurements
- **chart_itemid**: Identifier for charted measurement (foreign key to D_ITEMS)
- **chart_value**: Numeric value of the clinical measurement
- **chart_error**: Error flag for the measurement (if exists)
- **lab_itemid**: Identifier for laboratory test (foreign key to D_ITEMS)
- **lab_value**: Result value of the laboratory test

 4. Diagnostic Information
- **diagnose_code**: ICD-9 diagnosis code
- **seq_num**: Priority/sequence number of the diagnosis (1=primary)

 5. Treatment Information
- **input_itemid**: Identifier for input event (medications/fluids)
- **input_amount**: Quantity administered
- **input_rate**: Rate of administration
- **input_patientweight**: Patient weight at time of input (if recorded)

 6. Procedural Information
- **procedure_code**: ICD-9 procedure code performed during stay

transform your multiple rows per ICU stay into a single row format suitable for ML modeling while preserving both continuous values and procedure/diagnosis information

### B. Get one row by icu stay ID 

Handle LabEvents and InputEvents:

In [22]:
def efficient_pivot_topN(df, top_n=20):
    ## Step 1: Get top N most frequent lab and input items ##
    
    # Get top N lab_itemids
    top_lab_items = (
        df.filter("lab_itemid IS NOT NULL")
        .groupBy("lab_itemid")
        .count()
        .orderBy(F.desc("count"))
        .limit(top_n)
        .select("lab_itemid")
        .collect()
    )
    lab_itemids = [row['lab_itemid'] for row in top_lab_items]
    
    # Get top N input_itemids
    top_input_items = (
        df.filter("input_itemid IS NOT NULL")
        .groupBy("input_itemid")
        .count()
        .orderBy(F.desc("count"))
        .limit(top_n)
        .select("input_itemid")
        .collect()
    )
    input_itemids = [row['input_itemid'] for row in top_input_items]
    
    ## Step 2: Pivot only top N lab data ##
    lab_window = Window.partitionBy("icustay_id", "lab_itemid").orderBy("lab_value")
    
    lab_pivoted = (
        df.filter(F.col("lab_itemid").isin(lab_itemids))  # Only top N items
        .withColumn("rn", F.row_number().over(lab_window))
        .filter("rn = 1")
        .groupBy("icustay_id")
        .pivot("lab_itemid", lab_itemids)
        .agg(F.first("lab_value"))
        .fillna(-1)
    )
    
    ## Step 3: Pivot only top N input events ##
    input_window = Window.partitionBy("icustay_id", "input_itemid").orderBy(F.desc("amount"))
    
    input_pivoted = (
        df.filter(F.col("input_itemid").isin(input_itemids))  # Only top N items
        .withColumn("rn", F.row_number().over(input_window))
        .filter("rn = 1")
        .groupBy("icustay_id")
        .pivot("input_itemid", input_itemids)
        .agg(
            F.first("amount").alias("amount"),
            F.first("rate").alias("rate")
        )
        .fillna(-1)
    )
    
    ## Step 4: Get patientweight ##
    patientweight_df = (
        df.filter("patientweight IS NOT NULL")
        .groupBy("icustay_id")
        .agg(F.last("patientweight").alias("patientweight"))
    )
    
    ## Step 5: Get base columns ##
    base_cols = [
        "icustay_id", "admission_type", "admission_location", 
        "chart_itemid", "chart_value", "error", "lab_itemid", 
        "lab_value", "seq_num", "diagnose_code", "procedure_code",
        "first_careunit", "LOS"
    ]
    base_df = df.select(base_cols).distinct()
    
    ## Step 6: Join all components ##
    df_final = (
        base_df
        .join(lab_pivoted, "icustay_id", "left")
        .join(input_pivoted, "icustay_id", "left")
        .join(patientweight_df, "icustay_id", "left")
        .fillna(-1)
    )
    
    return df_final

In [32]:
# First repartition your input dataframe
df = df.repartition(200, "icustay_id")  # Repartition BEFORE processing
df.cache()  # Cache the repartitioned dataframe

print(f"Initial DF partitions: {df.rdd.getNumPartitions()}")  # Should show 200

# Then apply your pivot function
df_2 = efficient_pivot_topN(df, top_n=20)

# Cache the final result if you'll be using it multiple times
df_2.cache()
print(f"Final DF partitions: {df_2.rdd.getNumPartitions()}")  # May be different after operations


25/06/02 11:57:47 WARN CacheManager: Asked to cache already cached data.
ERROR:root:KeyboardInterrupt while sending command.][Stage 208:>  (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

25/06/02 12:01:15 WARN MemoryStore: Not enough space to cache rdd_883_1 in memory! (computed 3.4 GiB so far)
25/06/02 12:01:15 WARN BlockManager: Persisting block rdd_883_1 to disk instead.
25/06/02 12:37:22 WARN MemoryStore: Not enough space to cache rdd_883_1 in memory! (computed 2.3 GiB so far)
25/06/02 12:42:48 WARN MemoryStore: Not enough space to cache rdd_883_14 in memory! (computed 2.4 GiB so far)
25/06/02 12:42:48 WARN BlockManager: Persisting block rdd_883_14 to disk instead.
25/06/02 13:35:24 WARN MemoryStore: Not enough space to cache rdd_883_78 in memory! (computed 2.9 GiB so far)
25/06/02 13:35:24 WARN BlockManager: Persisting block rdd_883_78 to disk instead.


In [29]:
df_2 = efficient_pivot_topN(df, top_n=20)
df_2.cache()

ERROR:root:KeyboardInterrupt while sending command.][Stage 208:>  (0 + 1) / 1]  
Traceback (most recent call last):
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Handle Procedures and Diagnostics:

In [ ]:
def add_top_features(df_final, top_n=20):
    """Add top N procedure and diagnosis features to the final DataFrame"""
    
    # 1. TOP PROCEDURES ======================================================
    top_procedures = (
        df_final.filter("procedure_code IS NOT NULL")
        .groupBy("procedure_code")
        .count()
        .orderBy(F.desc("count"))
        .limit(top_n)
        .select("procedure_code")
        .collect()
    )
    top_procedure_codes = [row['procedure_code'] for row in top_procedures]
    
    # One-hot encode only top procedures
    procedure_encoded = (
        df_final
        .filter(F.col("procedure_code").isin(top_procedure_codes))
        .groupBy("icustay_id")
        .pivot("procedure_code", top_procedure_codes)
        .agg(F.first(F.lit(1)).otherwise(0))
        .fillna(0)
        .select(
            "icustay_id",
            *[F.col(code).alias(f"procedure_{code}") for code in top_procedure_codes]
        )
    )
    
    # 2. TOP DIAGNOSES ======================================================
    top_diagnoses = (
        df_final.filter("diagnose_code IS NOT NULL")
        .groupBy("diagnose_code")
        .count()
        .orderBy(F.desc("count"))
        .limit(top_n)
        .select("diagnose_code")
        .collect()
    )
    top_diagnosis_codes = [row['diagnose_code'] for row in top_diagnoses]
    
    # Importance-weighted diagnosis encoding (only for top diagnoses)
    window_spec = Window.partitionBy("icustay_id", "diagnose_code").orderBy("seq_num")
    
    diag_pivoted = (
        df_final
        .filter(F.col("diagnose_code").isin(top_diagnosis_codes))
        .withColumn("importance", F.lit(1)/F.col("seq_num"))
        .withColumn("rn", F.row_number().over(window_spec))
        .filter(F.col("rn") == 1)
        .groupBy("icustay_id")
        .pivot("diagnose_code", top_diagnosis_codes)
        .agg(F.first("importance"))
        .fillna(0)
    )
    
    # Rename diagnosis columns
    for col in diag_pivoted.columns:
        if col != "icustay_id":
            diag_pivoted = diag_pivoted.withColumnRenamed(col, f"diag_{col}")
    
    # 3. JOIN ALL FEATURES ==================================================
    df_final_with_features = (
        df_final
        .join(procedure_encoded, "icustay_id", "left")
        .join(diag_pivoted, "icustay_id", "left")
        .fillna(0, subset=(
            [f"procedure_{code}" for code in top_procedure_codes] +
            [f"diag_{code}" for code in top_diagnosis_codes]
        ))
    )
    
    return df_final_with_features

ERROR:root:KeyboardInterrupt while sending command.][Stage 196:>  (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/barbara/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "200")  
df_2 = df_2.repartition(20, "icustay_id")

df_final = add_top_features(df_2, top_n=20)
df_final.cache()

In [ ]:
df_final.show(5)

### C. Every entry has to be numeric

See data types of each column

In [ ]:
df_ready.schema

### D. Scale features

## 3. Target variable preparation (if categorical)

In [ ]:
label_indexer = StringIndexer(inputCol=target_col, outputCol="label")

## 4. Create Pipeline

In [ ]:
pipeline_stages = indexers + [assembler, scaler, label_indexer]
preprocessing_pipeline = Pipeline(stages=pipeline_stages)

In [ ]:
processed_data = preprocessing_pipeline.fit(df).transform(df)

## 5. Train-Test Split

In [ ]:
train_data, test_data = processed_data.randomSplit([0.7, 0.3], seed=42)
print(f"Training count: {train_data.count()}")
print(f"Test count: {test_data.count()}")

## 6. Model Training

In [ ]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Train model
model = rf.fit(train_data)

## 7. Predictions

In [ ]:
predictions = model.transform(test_data)

## 8. Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="label")
auc = evaluator.evaluate(predictions)
print(f"Test AUC = {auc}")

In [ ]:
# For multiclass classification
# evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print(f"Test Accuracy = {accuracy}")

## 9. Close Session

In [ ]:
spark.stop()